# Combined Evaluation Notebook

This notebook computes:

1. **Main evaluation (ISO/IEC 27002 & 27017)**
   - Metrics: Consistency, Completeness, Correctness, Generalizability, Precision
   - Method mapping:
     - 27002 → Method1 = SSSL, Method2 = LLM
     - 27017 → Method1 = KNN (derived), Method2 = LLM

2. **Other evaluation (Query comparison)**
   - Method1 = LLM, Method2 = Similarity
   - Computes averages and winner per Query


In [ ]:
%pip install openpyxl

In [ ]:
import re
import numpy as np
import pandas as pd

FILE = r'C:\Users\anoureldin\Downloads\Measuring Semantic Label Accuracy and Retrieval Precision (Responses) (1).xlsx'
SHEET = 'Form Responses 1'

df = pd.read_excel(FILE, sheet_name=SHEET)


## 1️⃣ MAIN EVALUATION

In [ ]:
MAIN_COL_RE = re.compile(
    r'^(Q\d+)\.\s*(.*?)\s*\[Method\s*([12])\s*-\s*([A-Za-z]+)\]$',
    flags=re.IGNORECASE
)


def normalize_header(col):
    return re.sub(r'\s+', ' ', str(col).replace('\r', ' ')).strip()


def parse_main_col(col):
    s = normalize_header(col)
    m = MAIN_COL_RE.match(s)
    if not m:
        return None
    qid = m.group(1).upper()
    qtext = m.group(2).strip()
    method_num = int(m.group(3))
    metric = m.group(4).title()
    return qid, qtext, method_num, metric


rating_cols, meta = [], []
for c in df.columns:
    parsed = parse_main_col(c)
    if parsed:
        rating_cols.append(c)
        meta.append((c, *parsed))

meta_df = pd.DataFrame(meta, columns=['col', 'QID', 'QuestionText', 'MethodNum', 'Metric'])
meta_df['order'] = np.arange(len(meta_df))
meta_df['IsQueryEval'] = False

question_order = (
    meta_df[['QuestionText', 'order']]
    .drop_duplicates(subset=['QuestionText'])
    .sort_values('order')
    .reset_index(drop=True)
)

if len(question_order) < 20:
    raise ValueError(f"Expected at least 20 unique questions, found {len(question_order)}.")

std1_questions = set(question_order.head(10)['QuestionText'])
meta_df['Standard'] = np.where(
    meta_df['QuestionText'].isin(std1_questions),
    'ISO/IEC 27002',
    'ISO/IEC 27017'
)


def method_name(row):
    if row['Standard'] == 'ISO/IEC 27002':
        return 'SSSL' if row['MethodNum'] == 1 else 'LLM'
    return 'KNN (derived from ISO/IEC 27017)' if row['MethodNum'] == 1 else 'LLM'


meta_df['Method'] = meta_df.apply(method_name, axis=1)

long = df[rating_cols].melt(var_name='col', value_name='Score').merge(meta_df, on='col', how='left')
long['Score'] = pd.to_numeric(long['Score'], errors='coerce')

means = long.groupby(['IsQueryEval', 'Standard', 'Method', 'Metric', 'QID'], as_index=False)['Score'].mean()


### Main table (Q1–Q10)

In [ ]:
try:
    from IPython.display import display
except Exception:
    def display(x):
        print(x)

metric_order = [
    m for m in ['Consistency', 'Completeness', 'Correctness', 'Generalizability', 'Precision']
    if m in means['Metric'].unique()
]
q_order = sorted(means['QID'].dropna().unique(), key=lambda x: int(re.sub(r'\D', '', str(x)) or 0))

main = means[means['IsQueryEval'] == False].pivot_table(
    index=['Standard', 'Method'],
    columns=['Metric', 'QID'],
    values='Score'
)

cols = [(m, q) for m in metric_order for q in q_order]
main = main.reindex(columns=pd.MultiIndex.from_tuples(cols))

# Average table: one value per metric (averaged across Q1..Q10), plus overall average.
main_avg = (
    means[means['IsQueryEval'] == False]
    .groupby(['Standard', 'Method', 'Metric'], as_index=False)['Score']
    .mean()
    .pivot_table(index=['Standard', 'Method'], columns='Metric', values='Score')
)
main_avg = main_avg.reindex(columns=metric_order)
main_avg['Overall_Avg'] = main_avg.mean(axis=1)

print('Main table (by Metric x QID):')
display(main)

print('Average table (by Metric + Overall):')
display(main_avg)


## 2️⃣ OTHER EVALUATION (Query winner comparison)

In [ ]:
QUERY_SCORE_RE = re.compile(
    r'^Score\s+the\s+relevance.*?Query\s*(\d+)\s*:.*?\[Method\s*([12])\]$',
    flags=re.IGNORECASE
)
QUERY_WINNER_RE = re.compile(
    r'^Select\s+which\s+method\s+performs\s+better\s+overall.*?Query\s*(\d+)\s*:',
    flags=re.IGNORECASE
)

query_score_cols = {}
winner_cols = {}

for c in df.columns:
    s = normalize_header(c)
    m = QUERY_SCORE_RE.match(s)
    if m:
        qnum = int(m.group(1))
        method_num = int(m.group(2))
        query_score_cols.setdefault(qnum, {})[method_num] = c
        continue

    w = QUERY_WINNER_RE.match(s)
    if w:
        qnum = int(w.group(1))
        winner_cols[qnum] = c

print('Detected query score columns:', {q: sorted(v.keys()) for q, v in query_score_cols.items()})
print('Detected winner columns:', sorted(winner_cols.keys()))

records = []
for qnum in sorted(query_score_cols):
    cols = query_score_cols[qnum]
    if 1 not in cols or 2 not in cols:
        continue

    m1 = pd.to_numeric(df[cols[1]], errors='coerce')
    m2 = pd.to_numeric(df[cols[2]], errors='coerce')

    win_col = winner_cols.get(qnum)
    if win_col is not None:
        win = df[win_col].astype(str).str.strip().str.lower()
        llm_wins = win.str.contains(r'\bmethod\s*1\b|\b1\b', regex=True, na=False).sum()
        similarity_wins = win.str.contains(r'\bmethod\s*2\b|\b2\b', regex=True, na=False).sum()
    else:
        llm_wins = 0
        similarity_wins = 0

    records.append({
        'Query': f'Query {qnum}',
        'LLM_avg': m1.mean(),
        'Similarity_avg': m2.mean(),
        'LLM_wins': llm_wins,
        'Similarity_wins': similarity_wins,
    })

other_eval = pd.DataFrame(
    records,
    columns=['Query', 'LLM_avg', 'Similarity_avg', 'LLM_wins', 'Similarity_wins']
).sort_values('Query').reset_index(drop=True)

if other_eval.empty:
    print('No query-evaluation rows were parsed. Check QUERY_SCORE_RE / QUERY_WINNER_RE against headers.')
    other_eval['Winner'] = pd.Series(dtype='object')
else:
    other_eval['Winner'] = np.select(
        [
            other_eval['LLM_wins'] > other_eval['Similarity_wins'],
            other_eval['Similarity_wins'] > other_eval['LLM_wins'],
        ],
        ['LLM', 'Similarity'],
        default='Tie'
    )

other_eval


## 💾 Export results

In [ ]:
with pd.ExcelWriter('final_results.xlsx') as w:
    main.to_excel(w, sheet_name='Main_Eval')
    main_avg.to_excel(w, sheet_name='Main_Avg')
    other_eval.to_excel(w, sheet_name='Other_Eval', index=False)

print('Saved: final_results.xlsx')
